# Importações e funções auxiliares

In [2]:
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pygame
import seaborn as sns
import random
from collections import defaultdict

# our classes
from agent import Agent, QLearningAgent, MonteCarloAgent
from environment import Environment
from screen import Screen

# define environment
ACTION_SPACE = np.eye(3)
NUM_ACTIONS = 3
NUM_STATES = 2 ** 11

# Set options to activate or deactivate the game view, and its speed
pygame.font.init()

def plot_metrics(metrics, filepath=None):
    formatted_dict = {'episodes': [],
                      'metrics': [],
                      'results': []}

    n = len(metrics['episodes'])
    for i in range(n):
        episode = metrics['episodes'][i]
        score = metrics['scores'][i]
        reward = metrics['rewards'][i]

        formatted_dict['episodes'].append(episode)
        formatted_dict['metrics'].append('score')
        formatted_dict['results'].append(score)

        formatted_dict['episodes'].append(episode)
        formatted_dict['metrics'].append('reward')
        formatted_dict['results'].append(reward)

    df_metrics = pd.DataFrame(formatted_dict)
    sns.lineplot(data=df_metrics, x='episodes', y='results', hue='metrics')
    if filepath is None:
        plt.show()
    else:
        plt.savefig(filepath)


def decode_state(encoded_state):
    """
    Decode a binary representation of a state into its decimal base;
    
    encoded_state: an array of 0s and 1s representing a binary value
    
    return: decimal value
    """
    decoded = ''
    for s in encoded_state:
        decoded += str(s)

    return int(decoded, 2)


def decode_action(encoded_action):
    if isinstance(encoded_action, np.ndarray):
        return encoded_action.argmax()
    return encoded_action

def default_reward(env):
    """
    Return the reward.
    The reward is:
        -10 when Snake crashes.
        +10 when Snake eats food
        0 otherwise
    """

    reward = 0
    if env.game.crash:
        reward = -10
    elif env.player.eaten:
        reward = 10

    return reward

pygame 2.0.0 (SDL 2.0.12, python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
import math
from scipy.spatial import distance

def reward_euclidean(env):
    player_pos = [env.player.x, env.player.y]
    food_pos = [env.food.x_food, env.food.y_food]
    dist = distance.euclidean(player_pos, food_pos)
    reward = 0
    if env.game.crash:
        reward = -10000
    elif env.player.eaten:
        reward = 10000
    else:
        if dist > 100:
            reward = (-1*dist)/10
        if dist <= 100:
            reward = (100-dist)/10

    return reward

# Monte Carlo

### Every Visit

In [4]:
def run_monte_carlo(agent: Agent, reward_function, episodes, display, speed, verbose=True, first_visit=False):
    # setting random seed
    random.seed(42)
    np.random.seed(42)

    if display:
        pygame.init()

    env = Environment(440, 440, reward_function)
    screen = Screen(env)

    episode = 0
    metrics = {'episodes': [],
               'scores': [],
               'rewards': []}

    returns_sum = {} #defaultdict(float)
    returns_count = {} #defaultdict(float)

    start = time.time()
    while episode < episodes:
        states_and_actions_visited = []

        if display:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    quit()

            screen.display()

        state1, done = env.reset()
        state1 = decode_state(state1)
        action1 = agent.choose_action(state1)
        episode_reward = 0
        while not done:
            # Getting the next state, reward
            state2, reward, done = env.step(action1)
            state2 = decode_state(state2)
            # Choosing the next action
            action2 = agent.choose_action(state2)

            # Learning the Q-value
            #decoded_action1 = decode_action(action1)
            #decoded_action2 = decode_action(action2)
            #agent.update(state1, state2, reward, decoded_action1, decoded_action2)

            state1 = state2
            action1 = action2
            episode_reward += reward

            if display:
                screen.display()
                pygame.time.wait(speed)

            states_and_actions_visited.append((state2, action2, reward))

            end = time.time()
            diff = end - start
            if diff > 600: # 10min
                break

        # Acabou o episódio, hora de aprender
        #sa_in_episode = set([(tuple(x[0]), x[1]) for x in states_and_actions_visited])
        G = 0
        for i, step in enumerate(states_and_actions_visited[::-1]):
            state, action, r = step
            G = agent.gamma*G + r

            do_update = True

            if first_visit:
                if state in [x[0] for x in states_and_actions_visited[::-1][len(states_and_actions_visited)-i:]]:
                    do_update = False

            if do_update:
                sa_pair = (step[0], decode_action(step[1]))

                if sa_pair in returns_sum:
                    returns_sum[sa_pair] += G
                else:
                    returns_sum[sa_pair] = G

                if sa_pair in returns_count:
                    returns_count[sa_pair] += 1.0
                else:
                    returns_count[sa_pair] = 1

                average = returns_sum[sa_pair] / returns_count[sa_pair]
                agent.Q[sa_pair[0], sa_pair[1] ] = average
                #agent.Q[state, action] = agent.Q[state, action] + 1/returns_count[sa_pair]

        # Incrementar episódios e ir guardando as nossas métricas
        episode += 1
        if verbose:
            print(f'Game {episode}      Score: {env.game.score}')

        mean_reward = episode_reward/episodes
        metrics['episodes'].append(episode)
        metrics['rewards'].append(mean_reward)
        metrics['scores'].append(env.game.score)

        end = time.time()
        diff = end - start
        print()
        if diff > 600: # 10min
            break


    return metrics

N0 = 1
gamma = 0.2

# define agent
monteCarloAgent = MonteCarloAgent(N0, gamma, NUM_STATES, NUM_ACTIONS, ACTION_SPACE)

start = time.time()
metrics = run_monte_carlo(monteCarloAgent, reward_function=reward_euclidean, episodes=300, speed=15, display=True)
end = time.time()

plot_metrics(metrics, filepath=None)

print('Run time:', (end-start), 'seconds')
print('Max. Score:', max(metrics['scores']))
print('Mean Last Scores:', np.mean(metrics['scores'][-50:]))

Game 1      Score: 0

Game 2      Score: 0

Game 3      Score: 1

Game 4      Score: 1

Game 5      Score: 0

Game 6      Score: 0

Game 7      Score: 1

Game 8      Score: 0

Game 9      Score: 2

Game 10      Score: 2

Game 11      Score: 2

Game 12      Score: 1

Game 13      Score: 5

Game 14      Score: 2

Game 15      Score: 3

Game 16      Score: 3

Game 17      Score: 7

Game 18      Score: 3

Game 19      Score: 6

Game 20      Score: 2

Game 21      Score: 7

Game 22      Score: 3

Game 23      Score: 5

Game 24      Score: 2

Game 25      Score: 3

Game 26      Score: 4

Game 27      Score: 3

Game 28      Score: 3

Game 29      Score: 4

Game 30      Score: 6

Game 31      Score: 3

Game 32      Score: 4

Game 33      Score: 1

Game 34      Score: 4

Game 35      Score: 2

Game 36      Score: 2

Game 37      Score: 4

Game 38      Score: 9

Game 39      Score: 1

Game 40      Score: 6

Game 41      Score: 10

Game 42      Score: 5

Game 43      Score: 2

Game 44      Score:

error: display Surface quit

### First Visit

In [26]:
N0 = 1
gamma = 0.2

# define agent
monteCarloAgent = MonteCarloAgent(N0, gamma, NUM_STATES, NUM_ACTIONS, ACTION_SPACE)

start = time.time()
metrics = run_monte_carlo(monteCarloAgent, reward_function=reward_euclidean, episodes=300, speed=0, display=True, first_visit=True)
end = time.time()

plot_metrics(metrics, filepath=None)

print('Run time:', (end-start), 'seconds')
print('Max. Score:', max(metrics['scores']))
print('Mean Last Scores:', np.mean(metrics['scores'][-50:]))

4.92422502470643
145.60219778561037
126.49110640673517
107.70329614269008
89.44271909999159
72.11102550927978
56.568542494923804
44.721359549995796
40.0
20.0
28.284271247461902
44.721359549995796
63.245553203367585
60.0
63.245553203367585
44.721359549995796
28.284271247461902
20.0
40.0
60.0
80.0
100.0
120.0
140.0
160.0
180.0
200.0
220.0
240.0
260.0
280.0
300.0
320.0
340.0
360.0
360.5551275463989
340.58772731852804
342.3448553724738
322.49030993194197
302.65491900843114
282.842712474619
263.0589287593181
243.3105012119288
223.60679774997897
203.9607805437114
184.39088914585776
164.92422502470643
145.60219778561037
126.49110640673517
107.70329614269008
89.44271909999159
72.11102550927978
56.568542494923804
44.721359549995796
40.0
20.0
28.284271247461902
44.721359549995796
63.245553203367585
60.0
63.245553203367585
44.721359549995796
28.284271247461902
20.0
40.0
60.0
80.0
100.0
120.0
140.0
160.0
180.0
200.0
220.0
240.0
260.0
280.0
300.0
320.0
340.0
360.0
360.5551275463989
340.587727318528

KeyboardInterrupt: 